In [1]:
from tensorflow import keras
import numpy as np

f_mnist = keras.datasets.fashion_mnist
(x_train_full, y_train_full), (x_test, y_test) = f_mnist.load_data()

x_train_full = np.reshape(x_train_full, (x_train_full.shape[0], x_train_full.shape[1] * x_train_full.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1] * x_test.shape[2]))

print(f"Training dataset: {x_train_full.shape}, {x_train_full.dtype}")
print(f"Testing dataset: {x_test.shape}, {x_test.dtype}")

x_train = x_train_full / 255.0
x_test = x_test / 255.0

y_train = y_train_full

2024-04-29 19:52:57.157096: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-29 19:52:57.160982: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 19:52:57.225200: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 19:52:57.225244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 19:52:57.226907: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Training dataset: (60000, 784), uint8
Testing dataset: (10000, 784), uint8


In [2]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [3]:
class_names = [
    'T-shirt',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'
]

class_names[y_train[0]]

'Ankle boot'

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from sklearn.decomposition import PCA
from sklearn.random_projection import GaussianRandomProjection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


classifying_models = {
    'Decision Tree': DecisionTreeClassifier(),
    'K-Neighbors': KNeighborsClassifier(),
    'Linear SVC': LinearSVC(dual=True),
    'Logistic Regression': LogisticRegression(solver='lbfgs')
}

dimensionality_methods = {
    'Original Dataset': lambda X, y: (x_train_scaled, y_train),
    'PCA': PCA(n_components=9),
    'Random Projection': GaussianRandomProjection(n_components=9),
    'LDA': LinearDiscriminantAnalysis(n_components=9)
}

classifying_analyze = {method: [] for method in dimensionality_methods}
classifying_analyze_time = {method: [] for method in dimensionality_methods}

In [5]:
import time


for model_name, model in classifying_models.items():
    for method_name, dimensionality_reducer in dimensionality_methods.items():
        model_instance = type(model).__name__
        if model_instance == 'LogisticRegression':
            model = LogisticRegression(solver='lbfgs', fit_intercept=True)

        print(f"Running {model_instance} with {method_name}\n")

        if method_name == "Original Dataset":
            start = time.time()
            model.fit(x_train_scaled, y_train)
            end = time.time()

            accuracy = model.score(x_test_scaled, y_test)

            classifying_analyze_time[method_name].append(end - start)
            classifying_analyze[method_name].append(accuracy)
            
            continue
            
        # Навчання моделі з використанням методу розмірності даних
        start = time.time()
        x_train_reduced = dimensionality_reducer.fit_transform(x_train_scaled, y_train)
        model.fit(x_train_reduced, y_train)
        end = time.time()

        # Оцінка результату
        x_test_reduced = dimensionality_reducer.transform(x_test_scaled)
        accuracy = model.score(x_test_reduced, y_test)

        # Збереження результатів
        classifying_analyze_time[method_name].append(end - start)
        classifying_analyze[method_name].append(accuracy)

Running DecisionTreeClassifier with Original Dataset

Running DecisionTreeClassifier with PCA

Running DecisionTreeClassifier with Random Projection

Running DecisionTreeClassifier with LDA

Running KNeighborsClassifier with Original Dataset

Running KNeighborsClassifier with PCA

Running KNeighborsClassifier with Random Projection

Running KNeighborsClassifier with LDA

Running LinearSVC with Original Dataset



/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Running LinearSVC with PCA



/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Running LinearSVC with Random Projection



...

/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Running LogisticRegression with Random Projection



/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Running LogisticRegression with LDA



/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
classifying_analyze, classifying_analyze_time

({'Original Dataset': [0.7926, 0.8533, 0.8281, 0.8439],
  'PCA': [0.7224, 0.7924, 0.6973, 0.7298],
  'Random Projection': [0.5393, 0.6744, 0.5001, 0.5302],
  'LDA': [0.7697, 0.8221, 0.8169, 0.8241]},
 {'Original Dataset': [70.14844083786011,
   0.07850313186645508,
   1091.2870614528656,
   26.28754234313965],
  'PCA': [7.694080591201782,
   6.659514904022217,
   75.00949311256409,
   9.511024951934814],
  'Random Projection': [3.3075530529022217,
   0.39260411262512207,
   112.97635984420776,
   5.784794330596924],
  'LDA': [33.80884099006653,
   19.51759958267212,
   63.99962854385376,
   23.38210105895996]})

In [7]:
classifying_analyze['Model'] = ['Decision Tree', 'K-Neighbors', 'Linear SVC', 'Logistic Regression']
classifying_analyze_time['Model'] = ['Decision Tree', 'K-Neighbors', 'Linear SVC', 'Logistic Regression']

In [8]:
import pandas as pd


data = classifying_analyze
df = pd.DataFrame.from_dict(data)

df.set_index('Model')

,Original Dataset,PCA,Random Projection,LDA
Model,,,,
Decision Tree,0.7926,0.7224,0.5393,0.7697
K-Neighbors,0.8533,0.7924,0.6744,0.8221
Linear SVC,0.8281,0.6973,0.5001,0.8169
Logistic Regression,0.8439,0.7298,0.5302,0.8241


In [9]:
data = classifying_analyze_time
df = pd.DataFrame.from_dict(data)

df.set_index('Model')

,Original Dataset,PCA,Random Projection,LDA
Model,,,,
Decision Tree,70.148441,7.694081,3.307553,33.808841
K-Neighbors,0.078503,6.659515,0.392604,19.517600
Linear SVC,1091.287061,75.009493,112.976360,63.999629
Logistic Regression,26.287542,9.511025,5.784794,23.382101
